## FILE TO EXPLORE

In [1]:
#filename='9x9_10games'
#filename='gogod_9x9_games'
#filename='KGS2001'
filename='KGS_10games'
#filename='KGS_100games'

In [2]:
import os
print(os.getcwd())

import sys
sys.path.append('my/path/to/module/folder')


/home/mario/Dropbox/PycharmProjects/ndsgo/data_processing


In [3]:
import sys

print(sys.path)

#add '/home/mario/Dropbox/PycharmProject/ndsgo/' to path
sys.path.append('/home/mario/Dropbox/PycharmProjects/ndsgo/')
print(sys.path)

['', '/home/mario/anaconda3/envs/tensorenv2/lib/python27.zip', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/plat-linux2', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-tk', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-old', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/lib-dynload', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/site-packages/IPython/extensions', '/home/mario/.ipython', 'my/path/to/module/folder']
['', '/home/mario/anaconda3/envs/tensorenv2/lib/python27.zip', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7', '/home/mario/anaconda3/envs/tensorenv2/lib/python2.7/plat-linux2', '/home/mario/anaconda3/envs/tensorenv2/l

## READ PROPERTIES FILE

In [4]:
with open(filename+'.prop','r') as f:
    
    class Properties():
        pass
    properties=Properties
    properties.ind_num_examples=[int(c) for c in f.readline().strip().split(',')]
    properties.train_num_examples=sum(properties.ind_num_examples[:-1])
    properties.test_num_examples=int(properties.ind_num_examples[-1])
    properties.total_num_examples=properties.train_num_examples+properties.test_num_examples
    properties.num_channels=int(f.readline().strip())
    properties.boardsize=int(f.readline().strip())
print(properties.ind_num_examples,properties.total_num_examples,properties.train_num_examples,
      properties.test_num_examples,properties.num_channels,properties.boardsize)

([286, 286, 286, 286, 286, 284], 1714, 1430, 284, 4, 19)


### TRAIN AND TEST ATTRIBUTES AND SIZES

In [5]:
import gzip
import struct
    
        
total_images=0
total_bytes=0
for s in ['train','test']:   
    with gzip.open(filename+'-'+s+'-labels-idx1-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        data=bytestream.read()
        bytes_label=len(data)
    
    with gzip.open(filename+'-'+s+'-vectors-idx4-ubyte.gz') as bytestream:
        magic_number=bytestream.read(4)
        num_images=struct.unpack('>I',bytestream.read(4))[0]
        size_1=struct.unpack('>I',bytestream.read(4))[0]
        size_2=struct.unpack('>I',bytestream.read(4))[0]
        size_3=struct.unpack('>I',bytestream.read(4))[0]
        data=bytestream.read()
        bytes_vector=len(data)
    total_images+=num_images
    total_bytes+=bytes_label+bytes_vector+24
    print('{}-labels. bytes_label:{}'.format(s,bytes_label))
    print('{} BYTES. bytes_vector:{} bytes_label:{}'.format(s.upper(),bytes_vector,bytes_label))
    print(s,num_images,size_1,size_2,size_3,bytes_label+bytes_vector,float(bytes_vector)/(size_1*size_2*size_3))
    
    if num_images==bytes_vector/(size_1*size_2*size_3):
        print('{} CORRECT'.format(s))
    else:
        print('{} INCORRECT'.format(s))
print('total_images',total_images,'total_bytes',total_bytes)


with open(filename + '.bin','rb') as bytestream:
    data=bytestream.read()
    bin_total_bytes=len(data)
    bin_total_images=len(data)/(size_1*size_2*size_3+2)
    print('bin',size_1,size_2,size_3,bin_total_bytes,bin_total_images)

if total_images==bin_total_images:
    print('CORRECT')
else:
    print('INCORRECT')
    

train-labels. bytes_label:2916
TRAIN BYTES. bytes_vector:2102464 bytes_label:2916
('train', 1456, 4, 19, 19, 2105380, 1456.0)
train CORRECT
test-labels. bytes_label:520
TEST BYTES. bytes_vector:372552 bytes_label:520
('test', 258, 4, 19, 19, 373072, 258.0)
test CORRECT
('total_images', 1714, 'total_bytes', 2478500)
('bin', 4, 19, 19, 2478444, 1714)
CORRECT


## VIEW BOARDS IN SEQUENTIAL FILE

In [18]:
# explore bin file
from utils import Color
from board import Board
import utils

regs=[]
N=properties.boardsize
chunksize=N*N*properties.num_channels+2
with open(filename+'.bin') as f:
    for i in range(5):
        chunk=f.read(chunksize)
        if chunk:
            regs.append(chunk)
        else:
            break
board=Board(N)            
player_color=Color.WHITE
for line in regs:
    board,a=Board.get_board_and_move_from_register_str(N, line, player_color)
    print(a)
    board.play(player_color,utils.a2p(a,N))
    print(board)
    player_color=Board.opposite_color(player_color)
    #if player_color==Color.BLACK:
    #    player_color=Color.WHITE
    #else:
    #    player_color=Color.BLACK
    

78

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . . . . . . . . . . . . . . 
18 . . . . . . . . . . . . . . . . . . . 
17 . . . . . . . . . . . . . . . . . . . 
16 . . . . . . . . . . . . . . . X . . . 
15 . .(O). . . . . . . . . . . . . . . . 
14 . . . . . . . . . . . . . . . . . . . 
13 . . . . . . . . . . . . . . . . . . . 
12 . . . . . . . . . . . . . . . . . . . 
11 . . . . . . . . . . . . . . . . . . . 
10 . . . . . . . . . . . . . . . . . . . 
 9 . . . . . . . . . . . . . . . . . . . 
 8 . . . . . . . . . . . . . . . . . . . 
 7 . . . . . . . . . . . . . . . . . . . 
 6 . . . . . . . . . . . . . . . . . . . 
 5 . . . . . . . . . . . . . . . . . . . 
 4 . . . X . . . . . . . . . . . X . . . 
 3 . . . . . . . . . . . . . . . . . . . 
 2 . . . . . . . . . . . . . . . . . . . 
 1 . . . . . . . . . . . . . . . . . . . 
   A B C D E F G H J K L M N O P Q R S T 


AttributeError: class Board has no attribute 'opposite_color'

## BOARDS IN SHUFFLED FILE


In [15]:
from utils import Color
from board import Board
import utils

regs=[]
N=properties.boardsize
chunksize=N*N*properties.num_channels+2
with open('shuffled_'+filename+'.bin') as f:
    for i in range(20):
        chunk=f.read(chunksize)
        if chunk:
            regs.append(chunk)
        else:
            break
board=Board(N)            
player_color=Color.WHITE
for line in regs:
    board,a=Board.get_board_and_move_from_register_str(N, line, player_color)
    print(a)
    board.play(player_color,utils.a2p(a,N))
    print(board)
    #if player_color==Color.BLACK:
    #    player_color=Color.WHITE
    #else:
    #    player_color=Color.BLACK

214

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . O X . . . . . X O . . X . 
18 . . . . X O O X O O X X . O X X O X O 
17 . . . X . O X O O X O O O O . X X O . 
16 . . . X . O X X X X X X O . . X . O X 
15 . . X . X X O X . . X O O X . X X X O 
14 . X . . . . . . . O X X O . O O X O O 
13 . O X X . X X . . O O X O O . O O . . 
12 O . O X X O X . X X X . X X O . O . O 
11 . O O O X O X . O O O X . . X . . O . 
10 . . O X X O X O O X O O O . . X X X O 
 9 . O . O X O O X . X X O . O X . . . X 
 8 . . . O X(O). X . X . . O X X O X X . 
 7 . O O X X O . . O X O O O O O X . . . 
 6 . O . O . O X X X O O X X X X X . . . 
 5 . O O X O . O X O O X O . O . . . . . 
 4 . X . X . . O O X X X O O X X X . X . 
 3 . . X . X X X . O . O O X O . . O . . 
 2 . . X O X O O O . . . X X X . . . . . 
 1 . . X . O . . . . . X . . . . . . . . 
   A B C D E F G H J K L M N O P Q R S T 
310

Black: 0 captures | White: 0 captures | Komi: 6.5
19 . . . . . . X O . . . . . O X . . O . 
18 . . .